In [2]:
!pip install transformers

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizer, BertModel
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Dataset que devuelva SECUENCIAS
# antes cada getitem devolvía UN SOLO TOKEN,
# lo cual hace imposible usar padding, batching.
# Ahora se agrupaman por instancia_id.

class dataset_secuencias(Dataset):
    def __init__(self, df,cant_instancias=None):

        # Agrupamos por secuencia
        # Cada key = instancia_id
        # Cada value = dataframe con todos sus tokens

        if cant_instancias != None:
          df=df[df["instancia_id"]<=cant_instancias]

        self.groups = df.groupby("instancia_id")
        self.keys = list(self.groups.groups.keys())

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        instancia_id = self.keys[idx]
        group = self.groups.get_group(instancia_id)

        tokens = torch.tensor(group["token_id"].tolist(), dtype=torch.long)
        labels_cap = torch.tensor(group["capitalizacion"].tolist(), dtype=torch.long)
        labels_pi = torch.tensor(group["punt_inicial"].tolist(), dtype=torch.long)
        labels_pf = torch.tensor(group["pfinal"].tolist(), dtype=torch.long)

        return tokens, labels_cap, labels_pi, labels_pf

In [6]:
# collate_fn con padding
#  DataLoader necesita un modo de juntar
#  secuencias de distintos tamaños y
#  rellenar con padding para formar un batch.

def collate_fn(batch):
    # batch es una lista de tuplas: (tokens, cap, pi, pf)

    # Ordenamos por longitud (requerido por pack_padded_sequence)
    batch.sort(key=lambda x: len(x[0]), reverse=True)

    seqs = [item[0] for item in batch]
    caps = [item[1] for item in batch]
    pis = [item[2] for item in batch]
    pfs = [item[3] for item in batch]

    lengths = torch.tensor([len(s) for s in seqs], dtype=torch.long)
    max_len = lengths.max()

    # Padding manual
    pad_token = 0
    pad_label = -100  # Ignorado por CrossEntropyLoss

    padded_seqs = torch.full((len(batch), max_len), pad_token, dtype=torch.long)
    padded_cap = torch.full((len(batch), max_len), pad_label, dtype=torch.long)
    padded_pi = torch.full((len(batch), max_len), pad_label, dtype=torch.long)
    padded_pf = torch.full((len(batch), max_len), pad_label, dtype=torch.long)

    for i in range(len(batch)):
        L = lengths[i]
        padded_seqs[i, :L] = seqs[i]
        padded_cap[i, :L] = caps[i]
        padded_pi[i, :L] = pis[i]
        padded_pf[i, :L] = pfs[i]

    return padded_seqs, padded_cap, padded_pi, padded_pf, lengths

In [7]:
# Implementacion de la RNN

class Uni_RNN(nn.Module):
    # hidden1: 256-384 según tu config
    # hidden2: 128-256 según tu config
    def __init__(self, hidden1=512, hidden2=256):
        super().__init__()

        bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.embedding_matrix = bert.embeddings.word_embeddings.weight
        self.embedding_dim = self.embedding_matrix.shape[1]

        self.LSTM = nn.LSTM(self.embedding_dim, hidden1, batch_first=True)
        self.LSTM2 = nn.LSTM(hidden1, hidden2, batch_first=True)

        self.fc_cap = nn.Linear(hidden2, 4)   # MULTICLASE
        self.fc_pinicial = nn.Linear(hidden2, 2)   # (2 clases)
        self.fc_pfinal = nn.Linear(hidden2, 4)   # MULTICLASE

    def forward(self, seqs, lengths, inferencia=False):

        emb = self.embedding_matrix[seqs]

        packed = nn.utils.rnn.pack_padded_sequence(
            emb, lengths.cpu(), batch_first=True, enforce_sorted=True
        )

        data, _ = self.LSTM(packed)
        data, _ = self.LSTM2(data)

        final, _ = nn.utils.rnn.pad_packed_sequence(data, batch_first=True)

        output_cap = self.fc_cap(final)
        output_punt_inicial = self.fc_pinicial(final)
        output_punt_final = self.fc_pfinal(final)

        if inferencia:
            output_cap = torch.softmax(output_cap, dim=-1)
            output_cap = torch.argmax(output_cap, dim=-1)

            output_punt_inicial = torch.softmax(output_punt_inicial, dim=-1)
            output_punt_inicial = torch.argmax(output_punt_inicial, dim=-1)

            output_punt_final = torch.softmax(output_punt_final, dim=-1)
            output_punt_final = torch.argmax(output_punt_final, dim=-1)

        return output_cap, output_punt_inicial, output_punt_final


In [8]:
# Cargar el dataframe y crear dataloader
df = pd.read_parquet("/content/drive/MyDrive/TP AA II/datos_modelo.parquet")

dataset = dataset_secuencias(df,30000)

train_size = int(0.8 * len(dataset))

valid_size = len(dataset) - train_size

train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
valid_loader  = DataLoader(valid_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)


In [9]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:",device)



Device: cuda


In [10]:
# Crear Modelo
model = Uni_RNN(
    hidden1=32,
    hidden2=16
)

model=model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

ce  = nn.CrossEntropyLoss(ignore_index=-100)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
# Training Loop

num_epoch=1 #poner en 3

for epoch in range(num_epoch):

    model.train()
    total_loss = 0

    for seqs, caps, pini, pfin, lengths in train_loader:

        seqs = seqs.to(device)
        caps = caps.to(device)
        pini = pini.to(device)
        pfin = pfin.to(device)
        lengths = lengths.to(device)

        out_cap, out_pini, out_pfin = model(seqs, lengths)

        # Pérdida capitalización (multiclase)
        loss_cap = ce(
            out_cap.reshape(-1, 4),
            caps.reshape(-1)
        )

        # Pérdida puntuación inicial
        loss_pini  = ce(
            out_pini.reshape(-1, 2),
            pini.reshape(-1)
        )

        # Pérdida puntuación final (multiclase)
        loss_pfin = ce(
            out_pfin.reshape(-1, 4),
            pfin.reshape(-1)
        )


        loss = loss_cap + loss_pini + loss_pfin

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()/len(train_loader)


    model.eval()
    val_loss = 0

    with torch.inference_mode():
        for seqs, caps, pini, pfin, lengths in valid_loader:

            seqs = seqs.to(device)
            caps = caps.to(device)
            pini = pini.to(device)
            pfin = pfin.to(device)
            lengths = lengths.to(device)

            out_cap, out_pini, out_pfin = model(seqs, lengths)

            # capitalización
            loss_cap = ce(
                out_cap.reshape(-1, 4),
                caps.reshape(-1)
            )

            # Pérdida puntuación inicial
            loss_pini  = ce(
            out_pini.reshape(-1, 2),
            pini.reshape(-1)
            )

            # puntuación final
            loss_pfin = ce(
                out_pfin.reshape(-1, 4),
                pfin.reshape(-1)
            )


            loss = loss_cap + loss_pini + loss_pfin
            val_loss += loss.item()/len(valid_loader)

    print(f"Epoch {epoch+1}/{num_epoch} | Train Loss = {total_loss:.4f} | Val Loss = {val_loss:.4f}")




Epoch 1/3 | Train Loss = 1.0933 | Val Loss = 0.8752
Epoch 2/3 | Train Loss = 0.8019 | Val Loss = 0.8100
Epoch 3/3 | Train Loss = 0.7388 | Val Loss = 0.7998


In [55]:
def decodificador(data,lengths,model):
  """Con los tokens del input y el modelo podemos reconstruir el texto """
  from statistics import mode
  pinicial, pfinal , cap= model(data,lengths,inferencia=True) #listas que me dicen que poner


  if len(data.shape) == 1:
    pinicial = pinicial.unsqueeze(0)
    pfinal = pfinal.unsqueeze(0)
    cap = cap.unsqueeze(0)


  tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
  tokens= [tokenizer.convert_ids_to_tokens(data[i][:lengths[i]]) for i in range(data.shape[0]) ]


  for elemento in range(data.shape[0]): # por cada elemento en el batch

    i=0

    pinicial_elemento= pinicial[elemento][:lengths[i]]
    pfinal_elemento = pfinal[elemento][:lengths[i]]
    cap_elemento = cap[elemento][:lengths[i]]
    token_elemento=tokens[elemento][:lengths[i]]

    texto_final =""
    palabra=""
    palabras_tokenizadas=[]
    captializacion_palabras=[] #lista de subcaptilizaciones
    subcapitalizacion=[]
    while i <len(token_elemento):

      if pinicial_elemento[i]<=0.5:
        apertura=""
      else:
        apertura="¿"

      if pfinal_elemento[i]==0:
        cierre=""
      elif pfinal_elemento[i]==1:
        cierre="?"
      elif pfinal_elemento[i]==2:
        cierre="."
      elif pfinal_elemento[i]==3:
        cierre=","

      #Si tiene ## se lo borro
      if token_elemento[i][:2]=="##":
        palabra+=apertura+token_elemento[i][2:]+cierre
        subcapitalizacion.append(cap_elemento[i])

      else:
        palabra+=apertura+token_elemento[i]+cierre
        subcapitalizacion.append(cap_elemento[i])



      if i != len(token_elemento)-1 and token_elemento[i+1][:2]!="##": #si el proximo token no es decontinuacion termine la palabra
        palabras_tokenizadas.append(palabra)
        palabra=""
        captializacion_palabras.append(subcapitalizacion)
        subcapitalizacion=[]

      elif i == len(token_elemento)-1:
        palabras_tokenizadas.append(palabra)
        captializacion_palabras.append(subcapitalizacion)
      i+=1


    texto_final =""
    for i in range(len(palabras_tokenizadas)):

      #procesamiento de captializaciones
      subcapitlizacion= captializacion_palabras[i]
      if mode(subcapitalizacion)==0:
        palabra_procesada=palabras_tokenizadas[i]

      elif mode(subcapitalizacion)==1:
        palabra_procesada=palabras_tokenizadas[i][0].upper()+palabras_tokenizadas[i][1:]

      elif mode(subcapitalizacion)==2:
        palabra_procesada=""
        for j in range(len(palabras_tokenizadas[i])):

          if j % 2 == 0:
            palabra_procesada+=palabras_tokenizadas[i][j]

          else:
            palabra_procesada+=palabras_tokenizadas[i][j].upper()

      else:
        palabra_procesada=palabras_tokenizadas[i].upper()


      if i == 0:
        texto_final+=palabra_procesada
      else:
        texto_final+=" "
        texto_final+=palabra_procesada

    print(texto_final)

In [13]:
datos_visualizacion=pd.read_parquet("/content/drive/MyDrive/TP AA II/visualizacion.parquet")
dataset_visualizacion=dataset_secuencias(df)
visualizacion_loader = DataLoader(dataset_visualizacion, batch_size=16, shuffle=False, collate_fn=collate_fn)

In [56]:
for seqs, _ , _, _, lengths in visualizacion_loader:

        seqs = seqs.to(device)
        lengths = lengths.to(device)

        decodificador(seqs,lengths,model)
        break


¿sus experimentos ¿sí ¿eso es lo que me asusta el día en que nos prometimos ¿me habló de sus experimentos lba a descubrir algo tan asombroso que le haría dudar de su propia cordura ¿tenía algo raro en la mirada un halo de misterio sus palabras me entusiasmaron ¿jam¿ás ¿he dudado de él ¿pero me preocupa ¿no puedo evitarlo y ahora esta carta no puedo seguir con esta incertidumbre ¿he de saberlo ¿vi¿ctor le has visto ¿sí hace unas tres semanas ¿le vi paseando ¿solo por el bosque a mí también me habló de su trabajo ¿le pregunté si podía visitar su laboratorio ¿me miró furioso y dijo que no dejaba entrar a nadie se comportó de una forma muy extraña ¿qué? podemos hacer ¿podría estar enfermo ¿no te preocupes lré a ver al ¿dr ¿wa¿ld¿man fue profesor de ¿hen¿ry en la facultad ¿tal vez él pueda aclararme algo vi¿ctor eres un encanto ¿sabe¿s que por ti llegar¿ía al fin del mundo ¿no me gustaría ¿te aprecio demasiado o¿jal¿á fuera así
¿ten¿go miedo la he leído una y otra vez ¿pero no entiendo ¿ni 

In [30]:
seqs[4].shape

torch.Size([239])